In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor # Algorithm
from sklearn.impute import SimpleImputer # Way to get rid of NAN values
from sklearn.model_selection import train_test_split #auto split training and testing data
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read csv
training_file_path = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
testing_file_path = "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"
training_df = pd.read_csv(training_file_path)
testing_df = pd.read_csv(testing_file_path)

In [ ]:
#Clean and Preprocess Data
X = training_df.drop(["SalePrice"],axis=1)
y = training_df.SalePrice
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)
X_valid = testing_df
object_cols = list((X_train.dtypes == 'object')[(X_train.dtypes == 'object')].index)

In [ ]:
# Categorical data
OH_encoder = OneHotEncoder(handle_unknown = 'ignore',  sparse_output = False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

num_X_train = X_train.drop(object_cols,axis = 1)
num_X_valid = X_valid.drop(object_cols,axis = 1)

OH_X_train = pd.concat([num_X_train,OH_cols_train],axis = 1)
OH_X_valid = pd.concat([num_X_valid,OH_cols_valid],axis = 1)

#ensure all are strings
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

In [ ]:
# drop NAN values
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(OH_X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(OH_X_valid))
imputed_X_train.columns = OH_X_train.columns
imputed_X_valid.columns = OH_X_valid.columns

In [ ]:
#Choose Model/Algorithm and predict
First_Model = RandomForestRegressor(n_estimators = 200,random_state = 3)
First_Model.fit(imputed_X_train,y_train)
predictions = First_Model.predict(imputed_X_valid)

In [ ]:
# Save to CSV
submission = testing_df[["Id"]]
submission["Id"]
submission["SalePrice"] = predictions

In [ ]:
submission.to_csv("submission.csv",index = None)